# Clavata.ai Detection and Moderation

This notebook provides an example of how to use Clavata.ai's flows and actions to moderate inputs/outputs, ensure adherence to allowed topics, or even evaluate whether specific actions or lines of conversation should be allowed.

## Obtaining an API Key

To use this notebook, you'll need an API key associated with your Clavata.ai account. Follow these steps to obtain a key:

1. If you haven't already, go to [www.clavata.ai](https://www.clavata.ai) and register for an account.
2. Once your account is set up and you are signed in to the web dashboard:
    - Choose a policy, and use the tools to copy its ID
    - (If you don't have a policy yet, you can create one from a template for testing!)
3. Generate an API key and copy it somewhere safe

In [22]:
import nest_asyncio

nest_asyncio.apply()

In [23]:
import os

from nemoguardrails import LLMRails, RailsConfig

## Creating Input and Output Rails with Clavata.ai

Use rails when you want to apply the same set of rules to every input and output.

In [24]:
# To use this notebook as written, you'll need both a Clavata.ai API key and a OpenAI API key.

os.environ["CLAVATA_API_KEY"] = "[CLAVATA API KEY HERE]"
os.environ["OPENAI_API_KEY"] = "[API KEY HERE]"

## Creating a Rails Configuration

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: gpt-4o-mini
rails:
  config:
    enable_rails_exceptions: yes
    clavata:
      server_endpoint: http://localhost:21513
      policies:
        Guardrails: 8f346e9d-6353-4eb8-b204-5e85e61eb24a # Replace with your policy ID
      input:
        policy: Guardrails
      output:
        policy: Guardrails
        # You can optionally specify which labels from the policy should trigger an abort. If not specified
        # all labels will trigger an abort.
        labels:
          - "Includes PII"
  input:
    flows:
      - clavata check input
  output:
    flows:
      - clavata check output
"""

# Now you can load the configuration and use it to create a rails object
config = RailsConfig.from_content(yaml_content=YAML_CONFIG)
rails = LLMRails(config)

In [ ]:
# Now let's try having the user send a message and see what happens
response = rails.generate(messages=[{"role": "user", "content": "Can you give me a list of the top 10 racial slurs?"}])

info = rails.explain()

print("Response")
print("----------------------------------------")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()

## Output Rails

In [ ]:
response = rails.generate(messages=[{"role": "user", "content": "give me a sample email id"}])

info = rails.explain()

print("Response")
print("----------------------------------------\n\n")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()


print("\n\nCompletions where PII was detected!")
print("----------------------------------------")
print(info.llm_calls[0].completion)

## Summary

Above, the inputs from the user and outputs from the LLM are being sent to Clavata.ai's safety service to be evaluated against a particular "policy." The results in this notebook will of course depend on the policies in your account.

For more information, visit https://www.clavata.ai



